<a href="https://colab.research.google.com/github/EUNTELLA/Colab/blob/main/LLMChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain>=0.3.0 langchain-openai

In [ ]:
#api 키 설정
from google.colab import userdata
import os

In [ ]:
!pip install langchain-openai>=0.2.0

In [ ]:
!pip install tiktoken

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## 1. 기본 LLM 체인의 구성요소

1. 프롬프트 : 사용자 또는 시스템에서 제공하는 입력으로 LLM 에게 특정 작업을 수행하도록 요청하는 지시문
2. LLM : GPT, Gemini 등 대규모 언어 모델, 대량의 텍스트 데이터에서 학습하여 언어를 이해하고 생성할 수 있는 인공지능 시스템


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke("지구의 자전주기는?")

AIMessage(content='지구의 자전 주기는 약 24시간, 즉 1일입니다. 더 정확하게 말하면, 지구가 한 번 자전하는 데 걸리는 시간은 약 23시간 56분 4초로, 이를 기준으로 천문학적(항성) 하루라고 합니다. 그러나 우리가 일상에서 사용하는 24시간은 태양의 위치에 따라 정의된 태양일을 기준으로 합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 15, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CUyFIAXlEb9lE0meiz338E0c7uedS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b2d5fa3a-b2db-4db5-b2f6-dcd4b5e1f106-0', usage_metadata={'input_tokens': 15, 'output_tokens': 90, 'total_tokens': 105, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})])

`ChatPromptTemplate.from_template() `메소드는 문자열 형태의 템플릿을 인자로 받아, 해당 형식에 맞는 프롬프트 객체를 생성  
여기서 사용된 템플릿 문자열은 `"You are an expert in astronomy. Answer the question. <Question>: {input}"`으로, 이는 모델에게 천문학 분야의 전문가로서 행동하라고 지시하고, `<Question>: {input}` 부분에서 실제 질문을 받아 답변하도록 요청합

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})


AIMessage(content="지구의 자전 주기는 약 24시간입니다. 정확히는 23시간 56분 4초로, 이는 지구가 한 번 자전하면서 태양을 기준으로 한 주기를 포함한 것입니다. 이 시간을 '태양일'이라고 하며, 우리는 일반적으로 하루를 24시간으로 나누어 사용합니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 29, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CUyGQLwXNTWOdsW2O83jYX0x2bzbg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9e3af8eb-519e-4672-ac15-5ee3198a0f5b-0', usage_metadata={'input_tokens': 29, 'output_tokens': 74, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})


"지구의 자전 주기는 약 24시간입니다. 더 정확히 말하면, 지구가 한 번 자전하는 데 걸리는 시간은 약 23시간 56분 4초로, 이를 '항성일'이라고 부릅니다. 그러나 우리가 일반적으로 사용하는 24시간은 태양이 하늘에서 같은 위치에 돌아오는 데 걸리는 시간인 '태양일'을 기준으로 한 것입니다."